# Cost of Plutocracy

The aim of this article is to analyze some of the biggest DAOs in the crypto space and see how many of them are influenced by token holders which own a disproportionate amount of tokens compared to most voters.

In [1]:
# sets up the pynb environment
import os
import sys

import pandas as pd
import numpy as np


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.data_processing.filters import get_quartile_by_vp
from libs.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)


The data used for this article came from data sources that specialize in providing data on DAO governance, such as DeepDAO and Snapshot. Coingecko was also used to find market data for DAO native tokens.

Using this data, two spreadsheets were compiled which act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last five proposals each DAO opened. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters of each proposal which have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
plutocracy_report_data = pd.read_excel(
    "../plutocracy_report.xls", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "../plutocracy_report_filtered.xls", sheet_name=None, engine="openpyxl"
)


With this, we can begin asking ourselves some questions. Such as: **How many whales have voted in the last five proposals for each of the DAOs we analyzed?**

In [3]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [4]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
        "avg vp for non-whales",
        "avg vp for whales",
        "avg cost of vote",
    ],
)


,# of whales,all voters,avg vp for non-whales,avg vp for whales,avg cost of vote
Uniswap,565,12120,1.284866,6.275766e+04,6.095784
ENS,74,1609,3.521251,3.356937e+04,14.895998
Lido,129,2396,2.996389,7.571993e+05,1.455704
Frax,1,27,245572.420746,3.806272e+06,10.240349
Decentraland,16,264,94.580925,7.613737e+04,0.749201
Curve Finance,8,181,6146.094560,2.509741e+06,2.485927
Radicle,1,28,213089.903084,2.830379e+06,4.079941
Euler,3,82,1050.633823,6.614156e+04,5.136914
Hop,48,944,267.985913,3.610917e+05,0.083825


Here we can clearly see that for the DAOs which we choose to analyze the top 5% of voters have a clear, disproportionate amount of voting power comparared to the average voter.

We can also gain some insight at the economic might that these whales hold for each DAO at the time of voting by taking a look at the average cost of each vote for each DAO's native token at the time these proposals were active.

---

We move on to taking at what DAO participation would look like without these whale's involvement. We do so by comparing the scores of each proposal and checking whether or not the outcome of the proposal was changed after filtering out whales.

We check whether or not a propsal's outcome has changed simply by checking if the largest vote choice score has changed after filtering out whales, more specifically, in python we do:
```python
has_changed_outcome = not unfiltered_winning_choice_index == filtered_winning_choice_index
```

In [5]:
score_diffrences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [6]:
score_diffrences_dfs = dict()

for score_diffrence in score_diffrences:
    for organization, data in score_diffrence.items():
        data: dict[str, list] = data
        items = data.items()
        score_diffrences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "type",
                "choices",
                "score_diffrences",
                "total_vp",
                "has_changed_outcome",
                "changed_outcome_winner"
            ],
        )
        score_diffrences_dfs[organization]["total_vp"] = score_diffrences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)


First, let's look at Uniswap

In [7]:
score_diffrences_dfs["Uniswap"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852,single-choice,"['Wormhole', 'LayerZero', 'Celer', 'deBridge']","[9546794.425500717, 8863809.305899736, 540.859...",23548080.599735759,False,Wormhole
0xa3c7247632d3a08e4e73a63867908d0cd4402f48fa673945f93e3197dee5450a,single-choice,"['Yes', 'No']","[20056524.265649345, 4927056.974342918]",24994560.158377878,False,Yes
0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c,single-choice,"['add Overnight Finance', 'add DeFiEdge', 'add...","[10054.585075893452, 2708025.0728700403, 3412....",2728492.255083876,True,add Overnight Finance
0x387b5e811e9673359b98729af5c7f14a50b7d0ea9b4dcf9278686b238932e25d,single-choice,"['Yes - Enact changes', 'No - No change']","[30508464.50667036, 34.17723847808492]",61343398.710826278,False,Yes - Enact changes


We observe that out of the last five proposals only one had their outcome changed after removing whales: The proposal to [add a Liquidity Mining Manager for the Optimism-Uniswap Liquidty Mining Program](https://snapshot.org/#/uniswap/proposal/0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c).

In this case, "Overnight Finance" would've been the chosen LMM if it weren't for whale interevention, instead "DeFiEdge" was chosen for this campaign. From the data, we can see that without whale intervention, DeFiEdge would've had ~2,708,025 less votes out of the total ~2,728,492 voting power for the entire proposal. That is ~99.3% of voting power allocated to choosing "DeFiEdge".

In [8]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,add Overnight Finance,add DeFiEdge,add none
Scores,14335.930887,2.710600e+06,3556.089173
Score Differeces,10054.585076,2.708025e+06,3412.851497


One highly contentious proposal whose outcome did not change after filtering out whales was [this temperature check to choose which Eth <> BNB bridge to use for Uniswap v3 governance](https://snapshot.org/#/uniswap/proposal/0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852). We see that even for the least popular choices, whales still contributed to the majority of the votes. We also observe that whales held the overwhelming majority of voting power for the runner up, "LayerZero", with 99.9% of votes being allocated to this choice. By comparison, "Wormhole" received 65.1% of voting power from whales.

In [9]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Wormhole,LayerZero,Celer,deBridge
Scores,1.467367e+07,8.871274e+06,660.398099,2474.327951
Score Differeces,9.546794e+06,8.863809e+06,540.859826,2258.136409


In [10]:
score_diffrences_dfs["ENS"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7,single-choice,"['For', 'Against', 'Abstain']","[1667210.963067633, 0.0, 137.12454090065714]",1670185.073618348,False,For
0xd7eff781be059513b5cd64d79e709abbbc653944c9a8c621dc051e7b42a405cb,single-choice,"['For', 'Against', 'Abstain']","[1666938.9315321625, 100.38682608595627, 141.9...",1670154.219044510,False,For
0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b,single-choice,"['For', 'Against', 'Abstain']","[1446509.1013934365, 208348.53159207577, 13519...",1671687.088797375,False,For
0xdaff050d30c77fe469da7eaa5b5bae0cb892f91a89c119367e47faf370667240,approval,"['nick.eth', 'james.eth', 'accessor.eth', 'sim...","[2319757.8152544936, 2319757.8152544936, 23197...",2325021.309782755,False,nick.eth
0xa245dc7264ae072620434996e1c66d82c64cefc5a2edde5114d691a8559d0b5f,approval,"['thecap.eth (Cap)', 'slobo.eth', 'limes.eth',...","[2278346.061608189, 2278346.0616081893, 227834...",2282885.783012460,False,slobo.eth


In [11]:
score_diffrences_dfs["Lido"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x3436b98afacec300ec2c0f79bb8dfd16685ececeeb7a45a18754d43b710179e8,single-choice,"['For', 'Against']","[55124323.664074376, 2048432.8880345668]",57176658.163439780,False,For
0x32f6f095dc4f7b97665b560781d6e6795da11f9c8218936a505575040038d70f,single-choice,"['For', 'Against']","[57163748.21209293, 7563.113010643607]",57175091.606986538,False,For
0x7ac2431dc0eddcad4a02ba220a19f451ab6b064a0eaef961ed386dc573722a7f,single-choice,"['For', 'Against']","[57686947.46411716, 78.13038422652139]",57690201.588526487,False,For
0x7f19fed19b66df88b45d64c22f9905fbf716839a5052fb79b10ee2ef1241645d,single-choice,"['YAY', 'NAY']","[54546195.85193226, 7525.505910950667]",54557517.318199627,False,YAY
0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b,single-choice,"['1. For 6 new oracles, 6/11 set', '2. For 4 n...","[92934.92264371048, 52723122.2034864, 4535.772...",52826368.429703690,True,"1. For 6 new oracles, 6/11 set"


In [12]:
score_diffrences_dfs["Frax"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x1272233285b3b4ce026173ec4e2a59d6c8daba3d64b41e67ba160cc526fc5c7d,weighted,"['For', 'Against']",[3805813.804935219],6086689.646356250,False,For
0x9cc997cf19f797919e40d87e17c7eed87aefa10279fbeef89761cb1717ac56b8,weighted,"['For', 'Against']",[3805819.97255508],6073404.242473515,False,For
0x02aa360090c273c1218b84fe4b25e2be1d6a0210be1419087832da3ac15c38eb,weighted,"['For', 'Against']",[3805825.454883847],6072772.098435019,False,For
0xe53269e838ab2ca0d8e09f8326d2a4ffb3425ecb1b39dfe89b72b7178b486614,weighted,"['For', 'Against']",[3805830.2519215182],6072784.283546080,False,For
0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a,weighted,"['For', 'Against']",[5792040.529852133],9970590.463647339,False,For


In [13]:
score_diffrences_dfs["Decentraland"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0xdfc9d87313c293cbde71adbed9d585ccd79d28ae1dde555f674613920c8856c8,single-choice,"['yes', 'no']","[115.0, 0]",142.356378308,False,yes
0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5,single-choice,"['Yes', 'No', 'Invalid question/options']","[26021.000638214886, 0.0, 0]",26949.339992801,False,Yes
0xbf2891a1e4490c0ba14e3897c239de06483a2758dff2c8769c9f7ce9f4dfae2d,single-choice,"['yes', 'no']","[888071.9501538787, 0.0]",904485.119040474,False,yes
0x6576a4e2a22df3660d64a5af2a0d6c4819aa86ec40d420f1f1a99359dbf174fc,single-choice,"['Unban', 'Ban', 'Invalid question/options']","[809843.173033201, 0.0, 9364.50282589451]",844087.296455739,False,Unban
0xf574f75a5308ba8a35b5f6d2f3135dd185c5757bb52b5a029da153f35b21e741,single-choice,"['yes', 'no']","[569993.4788284205, 49981.66360630109]",631070.299142578,False,yes


In [14]:
score_diffrences_dfs["Curve Finance"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5,single-choice,"['Yes', 'No']","[23462.994316173717, 511330.4724932205]",574553.874246682,True,Yes
0x43c689ef6b7b1772abc033cd87b7caa4159f0dc281f9a159e3930fb00b57d707,single-choice,"['Do Nothing', '2Pool + current 3Pool', '2Pool...","[0.0, 559204.7021513436, 2510541.702086272]",4272657.271177988,False,2Pool + current 3Pool + 4Pool
0xa11d4d1800b91746afe8ead922eb1d37225a32145adf30bbda0cbdba2ef97bd3,single-choice,"['Yes', 'No']","[26054297.07744241, 0]",26097304.209488451,False,Yes
0x026a69e5c9780479332d88fa9779a3cec5a9479d05cc4ca61b1fa9eb099caa73,single-choice,"['yes', 'no']","[1891.6266956075615, 0]",6893.077810757,False,yes


In [15]:
score_diffrences_dfs["Radicle"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0xf95b0e03c92f9f2df976cad6c5121383f4efc887162d0bf6b59cc550700bceec,single-choice,"['For', 'Against']","[2000000.0, 0]",4133689.798082654,False,For
0x646e92794f8ea0abb333ed264dc8d8a3df7ceae909186f88701b3237852e61fb,single-choice,"['Yes', 'No', 'Abstain']","[3730867.698436455, 0, 0]",6163639.615623794,False,Yes
bafkreia4tl25spzwtys4kfns3iiuj32n2wugyupa6adg6mlxwg77a3bzx4,single-choice,"['For', 'Make No Change', 'Abstain']","[2654186.698436455, 0, 0]",6922974.673405429,False,For
0x16400b2c86d6ebdea246372182f18a17394435d9e31f9749ebaad89e37f3eb25,basic,"['For', 'Against', 'Abstain']","[3090365.0190418432, 0, 0]",4971478.525081047,False,For
0x704025e577ff7e4e202985d342c30285c4d0d320a18c34b764aa9b4dd38fd8ec,basic,"['For', 'Against', 'Abstain']","[3090365.0190418432, 0, 0]",4946306.104718991,False,For


In [16]:
score_diffrences_dfs["Euler"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x8721e87559f34b71b5ffc51cb9717d90677259d7a2b3d1fd06524f86d3202745,single-choice,"['Delegate $SAFE to StableLab', 'Delegate $SAF...","[100000.0, 14950.714739293624, 0, 26366.0]",197217.723814994,True,Abstain
0xe16545bf8aa67399720dba057cf9d23f1e73caa8dad8463fef783a64c89c1837,single-choice,"['YES - Implement changes', 'NO- Do not Implem...","[332920.55799487286, 0]",388710.341413462,False,YES - Implement changes
0x3635964ebd6535eb25ea522b8aa5981942c264249185506bb1bd9474b7c9ca72,single-choice,"['Yes - make rETH collateral', 'No - do not ma...","[332920.55799487286, 0.0]",397650.270054811,False,Yes - make rETH collateral
0x4129f05639d2ca2f76cba392a625fa2ebe9b32964d29e7e45dc831a8792d8af9,single-choice,"['YES - implement the changes ', 'NO -do not i...","[54944.92856057068, 0.0]",105724.703030176,False,YES - implement the changes
0xeb9b8a930c53497494f0ba9af3b4411933b3aa2bbf845140f1645515763e8501,single-choice,"['YES - Implement the Changes ', 'NO - Do Noth...","[47147.733648946596, 291603.84325557924]",403627.500798613,True,YES - Implement the Changes


In [17]:
score_diffrences_dfs["Hop"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x29c0205077e5041a00b602c2e71a1b2ca5d84e2f7b4f02d2c7102948d4ffc783,single-choice,"['Yes', 'No', 'Abstain']","[11802995.848545816, 0.0, 0.0]",11985192.830100920,False,Yes
0xf1bf9327ad089d25e949b044d1673501acbeef96f1657d0e79b89fe365983595,single-choice,"['Yes', 'No', 'Abstain']","[13375900.370554667, 0.0, 0.0]",13563504.098747009,False,Yes
0x6b5136469fd7b1478c0e10b95e30e8645a1cea8bb49cabeed59a312264b518f0,single-choice,"['Yes', 'No', 'Abstain']","[13366384.657650102, 6729.757727668588, 0.0]",13556220.919344630,False,Yes
0x77dfa74f2fa447137b13d8e5a6faaa7c9f2d1a23f86e837fb6e0aea84902c82f,single-choice,"['Yes', 'No', 'Abstain']","[13371126.982221516, 6985.847465385696, 0.0]",13565274.220805701,False,Yes
0xdf8dbd41184c7c401ee326c62ca6e253a8fa856b825b7d9e9311ce0faea4a5ef,basic,"['For', 'Against', 'Abstain']","[12960574.180947237, 10677.532957239824, 87717...",13190658.527502781,False,For
